In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import loader, random, noise, csv

from PIL import Image
from Dijkstra.Dijkstra import *

## Data Generation Settings

#### Data Settings
**Shape**: Dimensions of the output image.<br>
**Output**: Output folder for the data.<br>
**Validation**: Wether to generate validation data or training data.<br>
**Image**: How many images to create.<br>

#### Noise Settings
Note: Noisemap will not be overitten if it already exists<br>
**Scale**: Noise scale.<br>
**Octaves**: How many times to apply the noise.<br>
**Persistence**: How much the noise amplitue changes for each octave.<br>
**Lacunarity**: How much the noise frequency changes for each octave.

In [13]:
shape = (256, 256)
output = "data/"
validation = False
images = 60000

scale = 100.0
octaves = 6
persistence = 0.5
lacunarity = 2.0

## Data Generation
### Helper Function

#### Get Height

Calculates the perlin noise height at a given point (i, j).<br>
**Return**: A float value between -1 and 1

In [3]:
def get_height(i, j):
    return noise.pnoise2(i/scale, j/scale, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=shape[0], repeaty=shape[1], base=0)

#### Generate Map

Calculates noise values for the entire grid with the dimensions given in the shape setting. The heightvalues are normalized to between 0 and 1.<br>
**Return**: A grid of dimensions (shape.x, shape.y) that contains normalized perline height noise.

In [4]:
def generate_map():
    grid = []
    for i in range(shape[0]):
        grid.append([])
        for j in range(shape[1]):
            n = get_height(i, j)
            n += 1
            n /= 2
            
            grid[i].append(Node(n, (i, j)))
            
    return grid

#### Get Node

Get node at point.<br>
**Return**: Node at point give.

In [5]:
def get_node(world, point):
    return world[point[0]][point[1]]

#### Get Random Points

Calculates a random start and stop inside the shape given in the settings.<br>
**Return**: Start and End point (Not node).

In [6]:
def get_random_points():
    start = (random.randrange(0, shape[1]), random.randrange(0, shape[1]))
    end = (random.randrange(0, shape[1]), random.randrange(0, shape[1]))
    
    return start, end

#### Create Image

Create a training image for DeepStar. The R axis contains the heightdata from the grid, G and B axies contains the start and stop point respectivly.<br>
**Return**: RGB training image fro DeepStar.

In [7]:
def create_image(grid):
    im = Image.new("L", shape)
    pixels = im.load()
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            intNoise = int(grid[i][j].value * 256)
            pixels[i, j] = intNoise
            
    return im

In [8]:
def load_image(img):
    world = []
    pixels = img.load()
    
    for x in range(shape[0]):
        world.append([])
        for y in range(shape[1]):
            if isinstance(pixels[x, y], int):
                value = pixels[x, y]
            else:
                value = pixels[x, y] if len(pixels[x, y]) == 1 else pixels[x, y][0]
                
            world[x].append(Node(value / 256, (x, y)))
            
    return world

## Create Files

If the necessary is not present in the targeted directory generate them.

In [15]:
world = generate_map()
edges = get_edges(world)

subfolder = "images"
if (validation):
    subfolder = "validation"

if (os.path.isfile(f'{output}{subfolder}/map.png')):
    img = Image.open(f'{output}{subfolder}/map.png')
    world = load_image(img)
    
if (not os.path.isfile(f'{output}{subfolder}/map.png')):
    img = create_image(world)
    img.save(f'{output}{subfolder}/map.png')
    
if (not os.path.isfile(f'{output}{subfolder}/data.csv')):
    with open(f'{output}{subfolder}/data.csv', 'w') as file: 
        writer = csv.writer(file)
        writer.writerow(["Start", "Stop", "Midpoint"])

## Generate Data

Generating the actual data and writing it to the csv. As the heightmap is always the same it does not need to write a image for each path and can store only the necessary values in a csv file. The path are calculated using dijkstra to make sure its the optimal path.

In [19]:
with open(f'{output}/{subfolder}/data.csv', 'a') as file:
    writer = csv.writer(file)
    for image in range(images):
        start, end = get_random_points()

        path = dijkstra(edges, start, end)
        try:
            path = get_path(path)
        except:
            continue
            
        if path is None:
            continue
            
        midpoint = get_path_midpoint(path)
        if len(path) < 11:
            midpoint = path[len(path) - 1]
        else:
            midpoint = path[10]
        
        writer.writerow([start, end, midpoint])
        print(image)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
28

KeyboardInterrupt: 